# I will be investigating the correlation to MLB Pitcher BMI and Velocity

## I will be looking at stats from last year where pitchers ahve at least 100 pitches in the season

In [15]:
#Import Necessary libraries
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [16]:
# Read CSV file
file_path = 'pitch_movement.csv' 
def read_csv_file(file_path):
    df = pd.read_csv(file_path)
    return df

In [17]:
# Display the first few rows of the DataFrame
def display_data(df):
    print("First few rows of the DataFrame:")
    print(df.head())
    return df  

# Call the functions
df = read_csv_file(file_path)
df = display_data(df)

First few rows of the DataFrame:
   year last_name, first_name  pitcher_id  team_name team_name_abbrev  \
0  2024       Gore, MacKenzie      669022  Nationals              WSH   
1  2024       Peralta, Freddy      642547    Brewers              MIL   
2  2024         Rodón, Carlos      607074    Yankees              NYY   
3  2024        Gausman, Kevin      592332  Blue Jays              TOR   
4  2024        Greene, Hunter      668881       Reds              CIN   

  pitch_hand  avg_speed  pitches_thrown  total_pitches  pitches_per_game  ...  \
0          L       96.0            1657           2994         18.481481  ...   
1          R       94.3            1650           3066         18.925926  ...   
2          L       95.4            1531           3102         19.148148  ...   
3          R       94.0            1487           2958         18.259259  ...   
4          R       97.6            1415           2563         15.820988  ...   

   league_break_z diff_z rise  pitcher_br

In [18]:
# Rename colum last_name, first_name to name
df = df.rename(columns={'last_name, first_name': 'name'})

print(df.head())

   year             name  pitcher_id  team_name team_name_abbrev pitch_hand  \
0  2024  Gore, MacKenzie      669022  Nationals              WSH          L   
1  2024  Peralta, Freddy      642547    Brewers              MIL          R   
2  2024    Rodón, Carlos      607074    Yankees              NYY          L   
3  2024   Gausman, Kevin      592332  Blue Jays              TOR          R   
4  2024   Greene, Hunter      668881       Reds              CIN          R   

   avg_speed  pitches_thrown  total_pitches  pitches_per_game  ...  \
0       96.0            1657           2994         18.481481  ...   
1       94.3            1650           3066         18.925926  ...   
2       95.4            1531           3102         19.148148  ...   
3       94.0            1487           2958         18.259259  ...   
4       97.6            1415           2563         15.820988  ...   

   league_break_z diff_z rise  pitcher_break_z_induced  pitcher_break_x  \
0           -13.7    1.6   11

In [19]:
def format_name(name):
    try:
        last, first = [x.strip() for x in name.split(',')]
        return f"{first} {last}"
    except:
        return name

print("Formatted names:")
df['name'] = df['name'].apply(format_name)
print(df.head())

Formatted names:
   year            name  pitcher_id  team_name team_name_abbrev pitch_hand  \
0  2024  MacKenzie Gore      669022  Nationals              WSH          L   
1  2024  Freddy Peralta      642547    Brewers              MIL          R   
2  2024    Carlos Rodón      607074    Yankees              NYY          L   
3  2024   Kevin Gausman      592332  Blue Jays              TOR          R   
4  2024   Hunter Greene      668881       Reds              CIN          R   

   avg_speed  pitches_thrown  total_pitches  pitches_per_game  ...  \
0       96.0            1657           2994         18.481481  ...   
1       94.3            1650           3066         18.925926  ...   
2       95.4            1531           3102         19.148148  ...   
3       94.0            1487           2958         18.259259  ...   
4       97.6            1415           2563         15.820988  ...   

   league_break_z diff_z rise  pitcher_break_z_induced  pitcher_break_x  \
0           -13.7 

In [ ]:
# I know need to add columns to the dataframe to get the pitchers height and weight. 
# To do this I will need to scrape the data from the MLB website.
import requests
from bs4 import BeautifulSoup
import re
import time
from unidecode import unidecode


In [ ]:
# Format "Last, First" → "First Last"
def format_name(name):
    last, first = [x.strip() for x in name.split(",")]
    return f"{first} {last}"

# Build guessed Baseball-Reference ID
def slug_br_id(first, last, suffix='01'):
    return f"/players/{last[0].lower()}/{last[:5].lower()}{first[:2].lower()}{suffix}.shtml"

# Try to guess the player URL
def find_br_url(raw_name):
    try:
        # Handle "Last, First" → "First Last" and remove accents
        full_name = format_name(raw_name)
        full_name = unidecode(full_name)
        first, last = full_name.split()

        for suffix in ['01', '02', '03']:
            path = slug_br_id(first, last, suffix)
            url = f"https://www.baseball-reference.com{path}"
            r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            if r.status_code == 200:
                return url

        # Fallback: search the site (in case ID guess fails)
        search_url = f"https://www.baseball-reference.com/search/search.fcgi?search={full_name.replace(' ', '+')}"
        r = requests.get(search_url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(r.text, 'html.parser')
        a = soup.select_one("div.search-item-name a")
        if a and '/players/' in a['href']:
            return 'https://www.baseball-reference.com' + a['href']
    except Exception as e:
        print(f"Error in find_br_url({raw_name}): {e}")
    return None

# Extract height and weight
def scrape_height_weight(url):
    try:
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if r.status_code != 200:
            return None, None
        soup = BeautifulSoup(r.text, 'html.parser')
        meta = soup.find('div', id='meta')
        if meta:
            text = meta.get_text()
            height_match = re.search(r'(\d+-\d+)', text)
            weight_match = re.search(r'(\d{3}) lb', text)
            height = height_match.group(1) if height_match else None
            weight = weight_match.group(1) if weight_match else None
            return height, weight
    except Exception as e:
        print(f"Error scraping {url}: {e}")
    return None, None

# Example usage (use your real CSV here)
names = ["Gore, MacKenzie", "Peralta, Freddy", "Rodón, Carlos", "Gausman, Kevin"]

for raw_name in names:
    print(f"\nSearching: {raw_name}")
    url = find_br_url(raw_name)
    print(f"→ URL: {url}")
    if url:
        height, weight = scrape_height_weight(url)
        print(f"   Height: {height}, Weight: {weight}")
    else:
        print("   → Not found.")
    time.sleep(1.5)



Searching: Gore, MacKenzie
→ URL: None
   → Not found.

Searching: Peralta, Freddy
→ URL: None
   → Not found.

Searching: Rodón, Carlos
→ URL: None
   → Not found.

Searching: Gausman, Kevin
→ URL: None
   → Not found.
